In [92]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as f
from pyspark.sql.types import *

In [2]:
! pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_312
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
data_path = '/media/daniel/Seagate Basic/spark_data/libros/' # en esta carpeta deben encontrarse los 4 ficheros

In [4]:
spark = SparkSession.builder.master("local[*]")\
          .appName("practica 2 Ecosistema Spark parte 3")\
          .getOrCreate()

21/12/25 11:13:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/25 11:13:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 1. Carga de dataframes

El primer paso consiste en cargar los tres dataframes. Para ello, se leen los ficheros csv con la API de Spark.

In [42]:
ratings = spark.read.csv(data_path+'BX-Book-Ratings.csv', sep=';', header=True, inferSchema=True)
ratings.schema

StructType(List(StructField(User-ID,IntegerType,true),StructField(ISBN,StringType,true),StructField(Book-Rating,IntegerType,true)))

In [44]:
books = spark.read.csv(data_path+'BX-Books.csv', sep=';', header=True, inferSchema=True)
books.schema

StructType(List(StructField(ISBN,StringType,true),StructField(Book-Title,StringType,true),StructField(Book-Author,StringType,true),StructField(Year-Of-Publication,IntegerType,true),StructField(Publisher,StringType,true),StructField(Image-URL-S,StringType,true),StructField(Image-URL-M,StringType,true),StructField(Image-URL-L,StringType,true)))

In [45]:
users = spark.read.csv(data_path+'BX-Users.csv', sep=';', header=True, inferSchema=True)
users.schema

StructType(List(StructField(User-ID,StringType,true),StructField(Location,StringType,true),StructField(Age,StringType,true)))

# 2. Consultas

### a. Lista de usuarios junto con el número de libros que han valorado

Se agrupa por usuario y se cuenta el número de reseñas que estan registradas en el fichero de reseñas.

In [59]:
ratings_count = ratings.groupBy('User-ID').count()

In [61]:
ratings_count.show(20)

+-------+-----+
|User-ID|count|
+-------+-----+
| 278515|    1|
|    463|    1|
|    496|    3|
|   1238|    1|
|   1591|    1|
|   1829|    1|
|   2366|    1|
|   3175|    1|
|   3918|    4|
|   4900|    4|
|   5300|   20|
|   5803|    3|
|   6336|    5|
|   6357|    2|
|   6397|    1|
|   6466|    2|
|   6654|    1|
|   7253|    5|
|   7340|    1|
|   7754|    1|
+-------+-----+
only showing top 20 rows



### b. Rating máximo recibido por cada editorial


En primer lugar se cruzan los datos de reseñas y de libros mediante un join. Despues se busca el máximo de las puntuaciones agregando por editorial.

In [68]:
joined = books.join(ratings, on='ISBN', how='left')
max_rating = joined.select('Publisher', 'Book-Rating').groupBy('Publisher').max()

max_rating.show()

+--------------------+----------------+
|           Publisher|max(Book-Rating)|
+--------------------+----------------+
|Harper Mass Marke...|              10|
|Houghton Mifflin ...|              10|
|Carroll &amp; Gra...|              10|
|      Celestial Arts|              10|
|Faith Publishing ...|               8|
|         Cleis Press|              10|
|Ullstein-Taschenb...|              10|
|Chicago Review Press|              10|
|    Adams Media Corp|              10|
|         Bertelsmann|               9|
|Orion Business Books|               1|
|        Book Pub. Co|               0|
|        Chosen Books|              10|
|         Hermetic Pr|              10|
|        Lorenz Books|              10|
| Research Press (IL)|               7|
|Editiones B, Grup...|               0|
|      Aqua Explorers|               9|
|           LPC Group|               9|
|Community Communi...|               8|
+--------------------+----------------+
only showing top 20 rows



### c. Nombre del autor que ha recibido más ratings

En este caso, reutilizamos el dataframe concatenado ("joined") del ejercicio anterior. Al haber hecho un left join, cada autor aparece tantas veces como haya aparecido el ISBN de su libro en el fichero de reseñas. Por tanto, basta hacer un recuento de cuántas veces aparece un autor en el dataframe "joined".

In [77]:
count_author = joined.select('Book-Author').groupBy('Book-Author').count().select('Book-Author', f.col('count').alias('ratings_count'))

count_author.show(20)

+--------------------+-------------+
|         Book-Author|ratings_count|
+--------------------+-------------+
|    Kate Young Caley|            1|
|       Deborah Dwork|            1|
|Woman's Day Magazine|            1|
|     R. A. Salvatore|          307|
|       J.P. Donleavy|            5|
|           Judy Ford|           12|
|DAWN PHD PRINCE-H...|            6|
|       Johnny Morris|            1|
|Key West Authors ...|            1|
|     Charles Kimball|            3|
|    Laurence Olivier|            5|
|        Steven Gould|           26|
|    F. Batmanghelidj|            5|
|Arthur Conan, Sir...|          186|
|       P. H. Russell|            1|
|           Bob Allen|            1|
|           Paul Ware|            1|
|  Caroline Moorehead|            6|
|        James Strong|            6|
|    Patrick Sebranek|            7|
+--------------------+-------------+
only showing top 20 rows



# 3. Window Functions

### a. ¿Cuál es el título del libro con mayor número de ratings para cada editorial?


Para este caso se usa una window function tal que:

- Particionamiento: Editorial
- Ordenamiento: Recuento de reseñas (descendente)

In [138]:
window = Window.partitionBy(joined_count.Publisher).orderBy(joined_count.ratings_count.desc())

Primero se calcula el número de reseñas para cada libro y editorial. Despues se particiona por editorial y ordena por número de reseñas. Se escogeran las primeras filas en el rankeo para dar el resultado pedido.

In [127]:
joined_count = joined.select('Publisher', 'Book-Title')\
                        .groupBy('Publisher', 'Book-Title')\
                        .count().select('Publisher', 'Book-Title', f.col('count').alias('ratings_count')) # calculo del numero de reseñas
result_window = joined_count.withColumn('rank', f.dense_rank().over(window)) # calculo del rankeo

result_filter_by_window = result_window.where('rank == 1')  # resultado final

Se puede apreciar que se calcula una clasificación (rank) en función del número de reseñas a nivel editorial de cada libro. En el siguiente ejemplo se observan los valores de ratings_count y rank de las filas de Adams Media Corp. Se ve que aaprecen ordenadas por el numero de reseñas y que si existen duplicados en el numero de reseñas, estas se rankean en la misma posicion.

In [128]:
result_window.show(10)

+-------------------+--------------------+-------------+----+
|          Publisher|          Book-Title|ratings_count|rank|
+-------------------+--------------------+-------------+----+
|A. &amp; M. Muchnik|Comuna Verdad (An...|            1|   1|
|   Adams Media Corp|365 Tv-Free Activ...|           10|   1|
|   Adams Media Corp|The Verbally Abus...|            7|   2|
|   Adams Media Corp|Show Biz Tricks f...|            4|   3|
|   Adams Media Corp|Jacob Marley's Ch...|            4|   3|
|   Adams Media Corp|The Complete Sing...|            3|   4|
|   Adams Media Corp| Mr. Cheap's Chicago|            2|   5|
|   Adams Media Corp|The Everything We...|            2|   5|
|   Adams Media Corp|Museum of Science...|            2|   5|
|   Adams Media Corp|Knock 'Em Dead: T...|            2|   5|
+-------------------+--------------------+-------------+----+
only showing top 10 rows



Y tambien se observa que para obtener el libro con mas reseñas de cada editorial basta filtrar por rank = 1 (continuando con el ejemplo anterior, observar la segunda fila del siguiente resultado)

In [129]:
result_filter_by_window.show(10)

+-------------------+--------------------+-------------+----+
|          Publisher|          Book-Title|ratings_count|rank|
+-------------------+--------------------+-------------+----+
|A. &amp; M. Muchnik|Comuna Verdad (An...|            1|   1|
|   Adams Media Corp|365 Tv-Free Activ...|           10|   1|
|     Aqua Explorers|Florida Shipwreck...|            1|   1|
|  Au diable Vauvert|               Rihla|            1|   1|
|        Bertelsmann| Abenteuer der Ferne|            2|   1|
|        Bertelsmann|Inseln der Südsee...|            2|   1|
|    Biosphere Press|Eating in: From t...|            1|   1|
|  Blue Harbor Press|The mariner's tri...|            1|   1|
|       Book Pub. Co| Spiritual midwifery|            2|   1|
|       Book Pub. Co|        Tofu cookery|            2|   1|
+-------------------+--------------------+-------------+----+
only showing top 10 rows



### b. ¿Cuál es la diferencia entre el número de ratings de cada libro y el número de ratings del libro con mayor número de ratings de la misma editorial?

Para este caso, se calcula el máximo de ratings con la misma partición que en el ajercicio anterior. Esta operación asignará a cada fila de la partición el rating máximo dentro de la partición. Así, se vuelve sencillo calcular dicha diferencia, ya que el problema se reduce a operar dos columnas del dataframe. 

In [146]:
ratings_diff = joined_count.withColumn('max_ratings_diff',
                                       f.max(joined_count.ratings_count).over(window) - joined_count.ratings_count)
                                        # maximo de la particion - rating de la fila

A continuación se observan algunos ejemplos del resultado. El máximo de la partición tiene diferencia 0 y el resto valores estrictamente mayores a 1. Si se pone el foco sobre Adams Media Corp, se ve que el máximo tiene valor 0 (10 reseñas) y el resto se ordenan de menor a mayor según su diferencia con respecto al máximo (10).

In [148]:
ratings_diff.show(20)

+-------------------+--------------------+-------------+----------------+
|          Publisher|          Book-Title|ratings_count|max_ratings_diff|
+-------------------+--------------------+-------------+----------------+
|A. &amp; M. Muchnik|Comuna Verdad (An...|            1|               0|
|   Adams Media Corp|365 Tv-Free Activ...|           10|               0|
|   Adams Media Corp|The Verbally Abus...|            7|               3|
|   Adams Media Corp|Show Biz Tricks f...|            4|               6|
|   Adams Media Corp|Jacob Marley's Ch...|            4|               6|
|   Adams Media Corp|The Complete Sing...|            3|               7|
|   Adams Media Corp| Mr. Cheap's Chicago|            2|               8|
|   Adams Media Corp|The Everything We...|            2|               8|
|   Adams Media Corp|Museum of Science...|            2|               8|
|   Adams Media Corp|Knock 'Em Dead: T...|            2|               8|
|   Adams Media Corp|Around the Americ

# 4. Conclusión

A modo de reflexión final, se observa que la API de Spark permite realizar consultas de manera cómoda e intuitiva. Por otro lado, tiene limitaciones en cuánto a consultas agregadas. Éstas se pueden subsanar de manera eficiente con el uso de Window Functions, las cuales simplifican el agregado de información tanto desde el punto de vista de la síntaxis como en eficiencia de computación.